In [4]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model # type: ignore
from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore
from sklearn.model_selection import train_test_split

In [5]:
base_dir = './hair-train' 
long_hair_dir = os.path.join(base_dir, 'long_hair')
short_hair_dir = os.path.join(base_dir, 'short_hair')


train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # 80% training, 20% validation


train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(48, 48),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(48, 48),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)




Found 8702 images belonging to 2 classes.
Found 2174 images belonging to 2 classes.


## Using Age and Gender prediction detection model

In [6]:
age_gender_model = load_model('Age_Sex_Detection.keras')
age_gender_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 48, 48, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 48, 48,    │        896 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 48, 48,    │          0 │ conv2d[0][0]      │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 48, 48,    │          0 │ dropout[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 24, 24,    │          0 │ activation[0][0]  │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 24, 24,    │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 24, 24,    │          0 │ conv2d_1[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 24, 24,    │          0 │ dropout_1[0][0]   │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 12, 12,    │          0 │ activation_1[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 12, 12,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 12, 12,    │          0 │ conv2d_2[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 12, 12,    │          0 │ dropout_2[0][0]   │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 6, 6, 128) │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 6, 6, 256) │    295,168 │ max_pooling2d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 6, 6, 256) │          0 │ conv2d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 6, 6, 256) │          0 │ dropout_3[0][0]   │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 3, 3, 256) │          0 │ activation_3[0][… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 2,050,760 (7.82 MB)

 Trainable params: 683,586 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,367,174 (5.22 MB)

In [7]:

def create_hair_length_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')  
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [8]:
hair_length_model = create_hair_length_model()
hair_length_model.fit(
    train_generator,
    validation_data=validation_generator,
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator),
    epochs=75  
)

c:\Users\adhit\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\adhit\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/75
272/272 ━━━━━━━━━━━━━━━━━━━━ 17s 55ms/step - accuracy: 0.9605 - loss: 0.1259 - val_accuracy: 0.9834 - val_loss: 0.0826
Epoch 2/75
272/272 ━━━━━━━━━━━━━━━━━━━━ 0s 91us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/75
  1/272 ━━━━━━━━━━━━━━━━━━━━ 22s 81ms/step - accuracy: 1.0000 - loss: 0.0150

c:\Users\adhit\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


272/272 ━━━━━━━━━━━━━━━━━━━━ 19s 69ms/step - accuracy: 0.9840 - loss: 0.0761 - val_accuracy: 0.9834 - val_loss: 0.0582
Epoch 4/75
272/272 ━━━━━━━━━━━━━━━━━━━━ 0s 46us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/75
272/272 ━━━━━━━━━━━━━━━━━━━━ 21s 78ms/step - accuracy: 0.9837 - loss: 0.0526 - val_accuracy: 0.9857 - val_loss: 0.0576
Epoch 6/75
272/272 ━━━━━━━━━━━━━━━━━━━━ 0s 59us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/75
272/272 ━━━━━━━━━━━━━━━━━━━━ 22s 82ms/step - accuracy: 0.9869 - loss: 0.0445 - val_accuracy: 0.9857 - val_loss: 0.0477
Epoch 8/75
272/272 ━━━━━━━━━━━━━━━━━━━━ 0s 127us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/75
272/272 ━━━━━━━━━━━━━━━━━━━━ 22s 82ms/step - accuracy: 0.9902 - loss: 0.0338 - val_accuracy: 0.9844 - val_loss: 0.0498
Epoch 10/75
272/272 ━━━━━━━━━━━━━━━━━━━━ 0s 72us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 11/75
272/272 ━━━━━━━━━━━━━━━━━━━━ 15s 56ms/step - accuracy: 0.9918 - loss: 0.0258 - val_accuracy: 0.98

In [9]:
hair_length_model.evaluate(validation_generator)


68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.9866 - loss: 0.1008


[0.10704796761274338, 0.9894204139709473]

In [10]:
def detect_hair_length(image):
    if image is None or image.size == 0:
        raise ValueError("Invalid image. The image is empty or could not be loaded.")
    img_resized = cv2.resize(image, (48, 48))  
    img_array = np.expand_dims(img_resized / 255.0, axis=0)
    prediction = hair_length_model.predict(img_array)
    return 'long' if prediction[0][0] > 0.5 else 'short'


def predict_age_gender(image):
    if image is None or image.size == 0:
        raise ValueError("Invalid image. The image is empty or could not be loaded.")
    
    img_resized = cv2.resize(image, (48, 48))  
    img_array = np.expand_dims(img_resized / 255.0, axis=0)  
    
    # Get the model prediction
    prediction = age_gender_model.predict(img_array)
    
    # Debugging output
    print(f"Model prediction output: {prediction}")

    if len(prediction) == 2:
        # Extracting and rounding the scalar age value
        age = int(np.round(prediction[1][0][0])) 

        # Extracting and rounding the scalar gender value
        gender_value = int(np.round(prediction[0][0][0]))  
        
        sex_f = ['Male', 'Female']
        
        gender = sex_f[gender_value]
        
    else:
        raise ValueError(f"Unexpected model output shape: {prediction.shape}")
    
    return age, gender


In [11]:
def classify_person(image):
    if image is None or image.size == 0:
        raise ValueError("Invalid image. The image is empty or could not be loaded.")
    age, gender = predict_age_gender(image)

    if 20 <= age <= 30:
        hair_length = detect_hair_length(image)
        if hair_length == 'long':
            return 'female'
        else:
            return 'male'
    else:
        return gender

In [12]:
image_path = '00076.jpg' # Replace with your test image path
image = cv2.imread(image_path)

try:
    result = classify_person(image)
    print(f"The detected person is classified as: {result}")
except ValueError as e:
    print(f"Error: {e}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
Model prediction output: [array([[0.]], dtype=float32), array([[23.413958]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
The detected person is classified as: female


In [13]:
hair_length_model.save('hair_length_detection_model.keras')